In [2]:
from selenium import webdriver
from PIL import Image
import time
DRIVER_PATH = 'chromedriver'
import requests
from bs4 import BeautifulSoup
import io
import hashlib

In [3]:
labels = {}
names = []
categories={}

In [4]:
import pandas as pd
 
# csv file name
filename = "table.csv"
 
data = pd.read_csv(filename)
words = data['Page'].tolist()
words

['Taj Mahal',
 'Burj Khalifa',
 'Statue of Liberty',
 'Great Wall of China',
 'Eiffel Tower',
 'Berlin Wall',
 'Machu Picchu',
 'Stonehenge',
 'Mount Rushmore National Memorial',
 'Colosseum',
 'Great Pyramid of Giza',
 'One World Trade Center',
 'Empire State Building',
 'White House',
 'Petra',
 'Large Hadron Collider',
 'Hagia Sophia',
 'Golden Gate Bridge',
 'Panama Canal',
 'Angkor Wat',
 'Big Ben',
 'Guantanamo Bay detention camp',
 'Buckingham Palace',
 'Christ the Redeemer (statue)',
 'Parthenon',
 'Disneyland',
 'Colossus of Rhodes',
 'New York Stock Exchange',
 'ADX Florence',
 'Notre-Dame de Paris',
 'Wembley Stadium',
 'Suez Canal',
 'Walt Disney World',
 'Westminster Abbey',
 'Channel Tunnel',
 'Forbidden City',
 'Willis Tower',
 'Windsor Castle',
 'Timbuktu',
 'Statue of Unity',
 'Taipei 101',
 'Arc de Triomphe',
 'Lighthouse of Alexandria',
 'John F. Kennedy International Airport',
 'Temple of Artemis',
 'Library of Alexandria',
 'Brooklyn Bridge',
 'Code of Hammurabi',


In [5]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=10):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
def persist_image(search_term,folder_path:str,url:str,temp):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        
        if image.width>300:
            basewidth = 300
            wpercent = (basewidth / float(image.size[0]))
            hsize = int((float(image.size[1]) * float(wpercent)))
            image = image.resize((basewidth, hsize), Image.ANTIALIAS)
            
        if image.height>300:
            baseheight = 300
            hpercent = (baseheight / float(image.size[1]))
            wsize = int((float(image.size[0]) * float(hpercent)))
            image = image.resize((wsize, baseheight), Image.ANTIALIAS)
        
        
        
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
        fil = hashlib.sha1(image_content).hexdigest()[:10] + '.jpg'
        labels[fil] = search_term
        names.append(fil)
        temp.append(fil)
        
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=20):
    target_folder = target_path

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=4)
    print(res) 
    temp=[]
    for elem in res:
        persist_image(search_term,target_folder,elem,temp)
    categories[search_term]=temp

In [8]:
import os

for i in range(len(words)):
    search = words[i]
    search_and_download( search_term = search, driver_path= DRIVER_PATH)

C:\Users\meena\AppData\Local\Temp\ipykernel_14640\3236804912.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")


Found: 100 search results. Extracting links from 0:100


C:\Users\meena\AppData\Local\Temp\ipykernel_14640\3236804912.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  actual_images = wd.find_elements_by_css_selector('img.n3VNCb')


Found: 20 image links, done!
{'https://media.vanityfair.com/photos/5e540c667dfff00008e5d14a/master/pass/Donald-Trump-at-the-taj-mahal.jpg', 'https://whc.unesco.org/uploads/thumbs/site_0252_0008-1200-630-20151104113424.jpg', 'https://cdn.britannica.com/86/170586-050-AB7FEFAE/Taj-Mahal-Agra-India.jpg', 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR1gL6A-mCNHbi2YKFM5w5CUxxGBX0i8BVWdw&usqp=CAU', 'https://www.designingbuildings.co.uk/w/images/b/b6/TajMahal1.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Taj_Mahal_in_March_2004.jpg/640px-Taj_Mahal_in_March_2004.jpg', 'https://th-thumbnailer.cdn-si-edu.com/NaExfGA1op64-UvPUjYE5ZqCefk=/fit-in/1600x0/filters:focal(1471x1061:1472x1062)/https://tf-cmsv2-smithsonianmag-media.s3.amazonaws.com/filer/b6/30/b630b48b-7344-4661-9264-186b70531bdc/istock-478831658.jpg', 'https://sustainabletravel.org/wp-content/uploads/Blog-Image-Taj-Mahal-Expectation-vs-Reality.png', 'https://www.thoughtco.com/thmb/l6mjGqVnMW8z53UcD86DE16ZG5c=

SUCCESS - saved https://luxeadventuretraveler.com/wp-content/uploads/2012/12/Luxe-Adventure-Traveler-Dubai-Burj-Khalifa-6.jpg - as ./images\997852f296.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/en/thumb/9/93/Burj_Khalifa.jpg/1200px-Burj_Khalifa.jpg - as ./images\5347d837b9.jpg
SUCCESS - saved https://img.veenaworld.com/wp-content/uploads/2021/11/Burj-Khalifa.jpg - as ./images\b7ed10d93d.jpg
SUCCESS - saved https://qph.cf2.quoracdn.net/main-qimg-207f763172989fb33952b1f61b2567df-lq - as ./images\8ef3a7361b.jpg
SUCCESS - saved https://images.adsttc.com/media/images/59eb/8a45/b22e/38dd/0500/0154/large_jpg/Burj_Khalifa_SOM_exterior_(c)_Nick_Merrick_for_Hedrich_Blessing_(9).jpg?1508608572 - as ./images\d653a0ede4.jpg
SUCCESS - saved https://thenational-the-national-prod.cdn.arcpublishing.com/resizer/P3KH6k13X1P0wePxJ7nir6191Hk=/800x0/filters:format(jpg):quality(70)/cloudfront-eu-central-1.images.arcpublishing.com/thenational/SVF3IG2NR7GNK2XRAPAIJE4ELE.jpg - as ./images\f5bbca

...

SUCCESS - saved https://pbs.twimg.com/profile_images/1351938473142448133/JQT93Cjo_400x400.jpg - as ./images\3300b41c58.jpg
SUCCESS - saved https://media.architecturaldigest.com/photos/6049345ec780349185c63395/master/pass/GettyImages-1223379545.jpg - as ./images\b2437b3a49.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQgk-DpKEebXldPAVG-_4Dd2TzZLpm28kOFjQ&usqp=CAU - as ./images\c37dc40223.jpg
SUCCESS - saved https://image.pbs.org/video-assets/pbs/white-house-inside-story/228965/images/mezzanine_417.jpg?focalcrop=1200x630x50x10&format=auto - as ./images\e9e810a80e.jpg
SUCCESS - saved https://media.istockphoto.com/photos/the-white-house-in-washington-dc-picture-id1300444386?b=1&k=20&m=1300444386&s=170667a&w=0&h=lEl2IpCzuBP2QUuzin3IC-fnCS1CISWhWRQujbmmlj8= - as ./images\31e27636e1.jpg
SUCCESS - saved https://www.whitehouse.gov/wp-content/uploads/2021/01/about_the_white_house.jpg - as ./images\efe9b06bd6.jpg
SUCCESS - saved https://yt3.ggpht.com/ytc/AKedOLSxIXP50k

SUCCESS - saved https://cds.cern.ch/images/CERN-EX-0510029-03/file?size=large - as ./images\d0f3660209.jpg
SUCCESS - saved https://imageio.forbes.com/specials-images/imageserve/5bd1fcffd88296682b9f2acb/Inside-the-LHC--protons-pass-each-other-at-299-792-455-m-s--3-m-s-shy-of-light-speed-/960x0.jpg?fit=bounds&format=jpg&width=960 - as ./images\5f142f2d32.jpg
SUCCESS - saved https://images.theconversation.com/files/438717/original/file-20211221-48250-esf86c.jpg?ixlib=rb-1.1.0&rect=0%2C142%2C1409%2C703&q=45&auto=format&w=668&h=324&fit=crop - as ./images\7fca61c265.jpg
SUCCESS - saved https://cdn.mos.cms.futurecdn.net/BdTwWBgmCnimmNuq57nbbm.jpg - as ./images\2df55bda59.jpg
SUCCESS - saved https://cdn.mos.cms.futurecdn.net/FR8sNkFA3Z96tMQkAKcYM7.jpg - as ./images\a7001817ec.jpg
SUCCESS - saved https://images.firstpost.com/wp-content/uploads/2019/09/201902-108_15sfs.jpg - as ./images\d842af5dc5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTzqiICeQgylpXKUEZzeKDCXik

SUCCESS - saved https://static.independent.co.uk/s3fs-public/thumbnails/image/2020/07/17/15/hagia-sophia-head.jpg?quality=75&width=1200&auto=webp - as ./images\c4613c53d4.jpg
SUCCESS - saved https://d1bv4heaa2n05k.cloudfront.net/user-images/1595583424182/shutterstock-505556230_main_1595583490077.jpeg - as ./images\545266aed5.jpg
Found: 100 search results. Extracting links from 0:100
Found: 20 image links, done!
{'https://s.hdnux.com/photos/01/24/07/71/22052416/6/1200x0.jpg', 'https://img.theculturetrip.com/wp-content/uploads/2019/01/rexfeatures_5886139ap.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a7/Vertigo_1958_trailer_Kim_Novak_at_Golden_Gate_Bridge.jpg/1200px-Vertigo_1958_trailer_Kim_Novak_at_Golden_Gate_Bridge.jpg', 'https://cdn.britannica.com/68/170868-050-8DDE8263/Golden-Gate-Bridge-San-Francisco.jpg', 'https://www.sunset.com/wp-content/uploads/readers-choice-best-bridge-golden-gate-getty-462144413-0819.jpg', 'https://www.worldtribune.org/wp-content/uploads/201

SUCCESS - saved https://img.etimg.com/thumb/width-1200,height-900,imgsize-71488,resizemode-1,msid-87353317/news/international/business/panama-canal-breaks-freight-record-despite-pandemic-crisis.jpg - as ./images\ec08a1338b.jpg
ERROR - Could not save https://gcaptain.com/wp-content/uploads/2019/08/panama_canal_speedlimit.jpeg - cannot identify image file <_io.BytesIO object at 0x000001C60177EF90>
SUCCESS - saved https://s29755.pcdn.co/wp-content/uploads/2020/12/Panama-Canal_July2020.jpg - as ./images\27dccfed09.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTemuYQk3RvW6wSk1y4b9OJYuDN9ityvIU4xA&usqp=CAU - as ./images\10cf73bf23.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSHVaDJoZl87HWBwShkYOWJU9stYCUWSRLNRg&usqp=CAU - as ./images\209729ce19.jpg
SUCCESS - saved https://i0.wp.com/www.circleofblue.org/wp-content/uploads/2015/01/AerialRenderingPacificAccess.jpg?ssl=1 - as ./images\a487a2bcdb.jpg
ERROR - Could not save https://www.prosp

KeyboardInterrupt: 

In [9]:
import json

with open('mylist1.txt','w') as f:
    f.write(json.dumps(names))
with open('mylist2.txt','w') as f:
    f.write(json.dumps(labels))
with open('mylist3.txt','w') as f:
    f.write(json.dumps(categories))   